In [1]:
from skimage.feature import hog
from skimage.io import imread
from skimage.transform import rescale
import skimage

from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import cv2
import os
import glob

X = []
y = []

load_images = [f for f in glob.glob(r'E:\ML\gender_ann\dataset' + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(load_images)

print(len(load_images))
for img in load_images:
    image = cv2.imread(img)
    image = cv2.resize(image, (96,96))
    image = tf.keras.preprocessing.image.img_to_array(image)
    # image = np.expand_dims(image, axis=0)
    X.append(image)
    typ = img.split(os.path.sep)[-2]
    if typ == "woman":
        typ = 0
    else:
        typ = 1  
    y.append([typ])

2307


In [3]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(2307, 96, 96, 3)
(2307, 1)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
print(X_test.shape)

(462, 96, 96, 3)


In [6]:
class RGB_to_Gray(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return np.array([skimage.color.rgb2gray(img) for img in X])
 
class Hogger(BaseEstimator, TransformerMixin):
    def __init__(self, y=None, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        def local_hog(X):
            return hog(X, orientations=self.orientations, pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block, block_norm=self.block_norm)
        return np.array([local_hog(img) for img in X])


In [7]:
grayer = RGB_to_Gray()
hoger = Hogger(pixels_per_cell=(8, 8),cells_per_block=(2,2),orientations=9,block_norm='L2-Hys')
scaler = StandardScaler()
 
X_train = grayer.fit_transform(X_train)
X_train = hoger.fit_transform(X_train)
X_train = scaler.fit_transform(X_train)

X_test=grayer.transform(X_test)
X_test=hoger.transform(X_test)
X_test = scaler.transform(X_test)

In [8]:
ann = tf.keras.models.Sequential()

In [9]:
ann.add(tf.keras.layers.Dense(units=36, activation='relu'))

In [10]:
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))

In [11]:
ann.add(tf.keras.layers.Dense(units=64, activation='relu'))

In [12]:
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))

In [13]:
ann.add(tf.keras.layers.Dense(units=12, activation='relu'))

In [14]:
ann.add(tf.keras.layers.Dense(units=18, activation='relu'))

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
58/58 [==============================] - 2s 4ms/step - loss: 0.5336 - accuracy: 0.7358
Epoch 2/100
58/58 [==============================] - 0s 3ms/step - loss: 0.1333 - accuracy: 0.9489
Epoch 3/100
58/58 [==============================] - 0s 3ms/step - loss: 0.0378 - accuracy: 0.9936
Epoch 4/100
58/58 [==============================] - 0s 3ms/step - loss: 0.0116 - accuracy: 0.9983
Epoch 5/100
58/58 [==============================] - 0s 3ms/step - loss: 0.0187 - accuracy: 0.9928
Epoch 6/100
58/58 [==============================] - 0s 3ms/step - loss: 0.0138 - accuracy: 0.9968
Epoch 7/100
58/58 [==============================] - 0s 3ms/step - loss: 0.0125 - accuracy: 0.9960
Epoch 8/100
58/58 [==============================] - 0s 3ms/step - loss: 0.0320 - accuracy: 0.9878
Epoch 9/100
58/58 [==============================] - 0s 3ms/step - loss: 0.0079 - accuracy: 0.9987
Epoch 10/100
58/58 [==============================] - 0s 3ms/step - loss: 0.0031 - accuracy: 0.9996
Epoch 11/

In [18]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [1 1]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [0 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 1]
 [0 1]
 [1 1]
 [1 1]
 [0 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[219  18]
 [ 18 207]]


0.922077922077922

In [20]:
import cvlib as cv
webcam = cv2.VideoCapture(0)    
classes = ['woman','man']

while webcam.isOpened():
    status, frame = webcam.read()
    face, confidence = cv.detect_face(frame)
    for idx, f in enumerate(face):
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
        face_crop = np.copy(frame[startY:endY,startX:endX])
        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = tf.keras.preprocessing.image.img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)
        face_crop = grayer.transform(face_crop)
        face_crop = hoger.transform(face_crop)
        face_crop = scaler.transform(face_crop)
        conf = ann.predict(face_crop)
        if conf > 0.5:
            label = "man: Hi bro"
        else:
            label = "woman: hello"
        Y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (145, 155, 49), 2)
    cv2.imshow("gender detection", frame)
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()